In [ ]:
import pandas as pd
import numpy as np
import panel as pn
from panel.template import FastListTemplate
pn.extension('tabulator')
pn.extension(sizing_mode='stretch_width')
import hvplot.pandas
import holoviews as hv

In [29]:
# Enable throttling for sliders to improve performance
pn.config.throttled = True

# Cache data to improve performance
if 'data' not in pn.state.cache.keys():
    df = pd.read_csv('./data/owid-energy-data.csv')
    df = df[df['year'] >= 1960]
    pn.state.cache['data'] = df.copy()
else: 
    df = pn.state.cache['data']

In [ ]:
coal_columns = [col for col in df.columns if col.startswith('coal')]
coal_columns

### (0) Data preprocessing

In [32]:
# Fill NAs with 0s and calculate gdp per capita for later use
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population']!=0, df['gdp']/df['population'],0)

In [33]:
# Make DF pipeline interactive
idf = df.interactive()

### (1) Coal consumption over time by continent

In [34]:
# Define panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=1960, end=2022, step=5, value=1970)
year_slider

IntSlider(end=2022, name='Year slider', sizing_mode='stretch_width', start=1960, step=5, value=1970)

In [35]:
# Radio buttons for coal measures
yaxis_coal = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['coal_consumption', 'coal_cons_per_capita'],
    button_type = 'success'
)

In [36]:
# Connect data pipeline with widgets here
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

# Create a subset of idf based on specified conditions
coal_pipeline = (
    idf[
        (idf.year >= year_slider) &  # Filter rows for year >= year_slider
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year','coal_production'])[yaxis_coal].mean() # Average of yaxis_coal for each country over time (year)
    .to_frame() # Convert result to dataframe
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True) # Discard old index and assign default integer index
)

In [38]:
# Plot the data
coal_plot = coal_pipeline.hvplot(x = 'year', by='country', y=yaxis_coal, line_width=2, title="Coal Consumption by Continent")

In [37]:
coal_pipeline.tail(10)

### (2) Heatmap - Coal production over time

In [39]:
# coal_table = coal_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
coal_heatmap = coal_pipeline.hvplot.heatmap(x='year', y='country', C='coal_production', cmap='Set3', title='Coal Production by Continent')

### (3) Coal vs GDP Scatterplot

In [40]:
# Create a subset of idf based on specified conditions
coal_gdp_pipeline = (
    idf[
        (idf.year == year_slider) & 
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['coal_consumption'].mean() # Average of coal_consumption for each country over time (year)
    .to_frame() # Convert result to df
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True) # Discard old index and assign default integer index
)

In [41]:
coal_gdp_pipeline

In [ ]:
coal_gdp_scatterplot = coal_gdp_pipeline.hvplot(x='gdp_per_capita', y='coal_consumption', by='country', size=80, kind="scatter", alpha=0.7, legend=False, height=500, width=500, title='Coal Consumption vs GDP Per Capita by Country')
coal_gdp_scatterplot

### (4) Bar chart with coal facts by continent

In [43]:
# Create a separate widget for the chart
yaxis_coal_facts = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['coal_cons_change_pct', 'energy_cons_change_pct', 'coal_share_energy'],
    button_type='success'
)

continents_excl = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']
coal_facts_bar_pipeline =  (
    idf[
        (idf.year == year_slider) & 
        (idf.country.isin(continents_excl))
    ]
    .groupby(['year', 'country'])[yaxis_coal_facts].sum()
    .to_frame() # Convert result to dataframe
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True) # Discard old index and assign default integer index
)    

In [ ]:
coal_facts_bar_plot = coal_facts_bar_pipeline.hvplot(kind='bar', x='country', y=yaxis_coal_facts, title='Coal Facts by Continent')
coal_facts_bar_plot

### (5) Repeat the steps above for oil

In [45]:
# Radio buttons for oil measures
df['oil_cons_per_capita'] = df['oil_consumption'] / df['population']
yaxis_oil = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['oil_consumption', 'oil_cons_per_capita'],
    button_type = 'success'
)

# Create a subset of idf based on specified conditions
oil_pipeline = (
    idf[
        (idf.year >= year_slider) &  # Filter rows for year >= year_slider
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year','oil_production'])[yaxis_oil].mean() # Average of yaxis_oil for each country over time (year)
    .to_frame() # Convert result to dataframe
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True) # Discard old index and assign default integer index
)
# Create plot, (table), and heatmap
oil_plot = oil_pipeline.hvplot(x = 'year', by='country', y=yaxis_oil, line_width=2, title="Oil Consumption by Continent")
# oil_table = oil_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
oil_heatmap = oil_pipeline.hvplot.heatmap(x='year', y='country', C='oil_production', cmap='Set3', title='Oil Production by Continent')

# Create a subset of idf based on specified conditions
oil_gdp_pipeline = (
    idf[
        (idf.year == year_slider) & 
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita', 'oil_cons_per_capita'])['oil_consumption'].mean() # Average of coal_consumption for each country over time (year)
    .to_frame() # Convert result to dataframe
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True) # Discard old index and assign default integer index
)
# Create a scatterplot
oil_gdp_scatterplot = oil_gdp_pipeline.hvplot(x='gdp_per_capita', y='oil_consumption', by='country', size=80, kind="scatter", alpha=0.7, legend=False, height=500, width=500,title='Oil Consumption vs GDP Per Capita by Country')
# Create a separate widget for the bar graph
yaxis_oil_facts = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['oil_cons_change_pct', 'energy_cons_change_pct', 'oil_share_energy'],
    button_type='success'
)
oil_facts_bar_pipeline =  (
    idf[
        (idf.year == year_slider) & 
        (idf.country.isin(continents_excl))
    ]
    .groupby(['year', 'country'])[yaxis_oil_facts].sum() # Bar graphs use sum to aggregate data
    .to_frame() # Convert result to dataframe
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True) # Discard old index and assign default integer index
)
oil_facts_bar_plot = oil_facts_bar_pipeline.hvplot(kind='bar', x='country', y=yaxis_oil_facts, title='Oil Facts by Continent')



### (6) Build the dashboard

In [ ]:
# Define your page classes with their content structure
class PageCoal:
    def __init__(self):
        self.content = pn.Column(
            "# Coal",
            pn.Row(
                pn.Column(yaxis_coal, coal_plot.panel(width=600), margin=(0, 25)),
                pn.Column(coal_heatmap.panel(width=700))
            ),
            pn.Row(
                pn.Column(coal_gdp_scatterplot.panel(width=600), margin=(0, 25)),
                pn.Column(yaxis_coal_facts, coal_facts_bar_plot.panel(width=700))
            )
        )

    def view(self):
        return self.content

class PageOil:
    def __init__(self):
        self.content = pn.Column(
            "# Oil",
            pn.Row(
                pn.Column(yaxis_oil, oil_plot.panel(width=600), margin=(0, 25)),
                pn.Column(oil_heatmap.panel(width=700))
            ),
            pn.Row(
                pn.Column(oil_gdp_scatterplot.panel(width=600), margin=(0, 25)),
                pn.Column(yaxis_oil_facts, oil_facts_bar_plot.panel(width=700))
            )
        )

    def view(self):
        return self.content

pages = {
    "Coal": PageCoal(),
    "Oil": PageOil(),
}

# Function to show the selected page
def show_page(page_instance):
    main_area.clear()
    main_area.append(page_instance.view())

# Button definitions to navigate between pages
page_coal_button = pn.widgets.Button(name="Coal", button_type="primary", width=250)
page_oil_button = pn.widgets.Button(name="Oil", button_type="primary", width=250)

# Button click callbacks
page_coal_button.on_click(lambda event: show_page(pages["Coal"]))
page_oil_button.on_click(lambda event: show_page(pages["Oil"]))

# Create the main area and display the first page
main_area = pn.Column(pages["Coal"].view())

# Create the Fast List Template and set the sidebar, main area, and styles
template = FastListTemplate(
    title="World Energy Dashboard",
    sidebar=[pn.pane.Markdown("#### This interactive panel provides insights into global energy trends and patterns over time. Select from the options below."),
             page_coal_button, 
             page_oil_button, 
             pn.pane.Markdown("## Settings"),
             year_slider],
    main=[main_area],
    accent_base_color="#d88888",
    header_background="#d8b088",
    sidebar_width=320
)

# Serve the Panel app
template.servable()
